In [740]:
import random
from sympy import isprime
from sympy import randprime
from sympy import legendre_symbol
from sympy import mod_inverse

class RabinSignature:
    
    def __init__(self, bits):
        self.bits = bits
    
    def get_keys(self):
        p = RabinSignature.generate_prime_with_mod_condition(4, 3, self.bits)
        q = RabinSignature.generate_prime_with_mod_condition(4, 3, self.bits)
        n = p * q
        return(p, q, n)
        
    @staticmethod
    def hash(m, p):
        m = int(''.join(format(ord(char), '08b') for char in m), 2)
        return pow(m, 2, p)
        
        
    @staticmethod
    def generate_prime_with_mod_condition(mod_div, mod_value, num_bits):
        while True:
            candidate = random.getrandbits(num_bits)
            candidate |= 1
            if candidate % mod_div == mod_value and isprime(candidate):
                return candidate
          
    @staticmethod
    def generate_random_prime(num_bits):
        while True:
            prime_number = randprime(2**(num_bits-1), 2**num_bits - 1)
            if prime_number:
                return prime_number
    
    def calc_x(self, m, p, q, n, u, h):
        lp = legendre_symbol(h, p)
        lq = legendre_symbol(h, q)
        xp = h ** (p // 2)
        xq = h ** (q // 2)
        while lp + lq != 2 and (xp <= 1 or xq <= 1):
            u = random.getrandbits(self.bits)
            h = RabinSignature.hash(m, u)
            lp = legendre_symbol(h, p)
            lq = legendre_symbol(h, q)
            xp = h ** (p // 2)
            xq = h ** (q // 2)
        mp = n / p
        mq = n / q
        yp = mod_inverse(pow(mp, -1), p)
        yq = mod_inverse(pow(mq, -1), q)
        x1 = xp * mp * yp % n 
        x2 = xq * mq * yq % n
        x = int(x1 + x2)
        return x
        
    def sign(self, m, public_keys):
        p, q, n = public_keys
        u = random.getrandbits(self.bits)
        while u <= 1:
            u = random.getrandbits(self.bits)
        h = RabinSignature.hash(m, u)
        x = self.calc_x(m, p, q, n, u, h)
        return (u, x)
        
    
    def verify(self, m, private_keys, public_keys):
        p, q, n = private_keys
        if (not isprime(p) or not isprime(q)):
            return False
        u, x = public_keys
        h = RabinSignature.hash(m, u)
        v = self.calc_x(m, p, q, n, u, h)
        return x == v
    

In [747]:
alg = RabinSignature(8)
t = []
for i in range(1000):
    private_keys = alg.get_keys()
    public_keys = alg.sign('hello world', private_keys)
    t.append(int(alg.verify('hello world', private_keys, public_keys)))
print(t)

[1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 